In [1]:
import utils
import os, re
import jieba
import numpy as np
from settings import Settings
from rule import Satisfy

In [2]:
setting = Settings()

loading file data\helper\size_log_plen50.pkl
finished loading data\helper\size_log_plen50.pkl


In [3]:
clean_dir = os.path.join('data', 'clean')
helper_dir = os.path.join('data', 'helper')
dataset_dir = os.path.join('data', 'dataset')

In [54]:
y_pred_start = utils.pickleLoader('y_pred_start_qanet.pkl')
y_pred_end = utils.pickleLoader('y_pred_end_qanet.pkl')
print(np.array(y_pred_start).shape)
print(np.array(y_pred_end).shape)

loading file y_pred_start_qanet.pkl
finished loading y_pred_start_qanet.pkl
loading file y_pred_end_qanet.pkl
finished loading y_pred_end_qanet.pkl
(64118, 50)
(64118, 50)


In [55]:
validation_set_raw = utils.pickleLoader(os.path.join(dataset_dir, 'validation_raw_plen50 (2).pkl'))

loading file data\dataset\validation_raw_plen50 (2).pkl
finished loading data\dataset\validation_raw_plen50 (2).pkl


In [56]:
print(len(validation_set_raw))

64118


In [57]:
idx = 18
print(validation_set_raw[idx][0][0])
print(validation_set_raw[idx][0][1])
print(validation_set_raw[idx][-1])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
{'answer': '澳门币', 'passage': {'passage_context_features': array([[0.        , 0.        , 0.5       , 0.5       ],
       [0.        , 0.        , 0.16666667, 0.16666667],
       [0.        , 0.        , 0.16666667, 0.        ],
       [0.        , 0.        , 0.16666667, 0.        ],
       [0.        , 0.        , 0.16666667, 0.        ],
       [1.        , 1.        , 0.33333333, 0.33333333],
       [1.        , 1.        , 0.66666667, 0.66666667],
       [1.        , 1.        , 0.66666667, 0.66666667],
       [0.        , 0.        , 0.41666667, 0.41666667],
       [0.        , 0.        , 0.5       , 0.5       ],
       [0.        , 0.        , 0.16666667, 0.16666667],
       

In [58]:
qidAnswers = {}
def read_question_answers():
    fnlist = ['data/qid_answer_expand', 'data/qid_answer_expand.train',  'data/qid_answer_expand.valid']
    for fn in fnlist:
        for tokens in utils.LoadCSV(fn):
            if len(tokens) != 3: continue
            qid = tokens[0]
            answers = tokens[2].split('|')
            qidAnswers[qid] = set(answers)
read_question_answers()

In [59]:
def get_answer(answer_rank):
    canswerScore = {}
    for j in answer_rank:
        canswer = j['answer']
        canswerScore[canswer] = canswerScore.get(canswer, 0) + j['score']**2
    cl = utils.toDouList(canswerScore)
    cl.sort(key=lambda x: x[1], reverse=True)
    return cl

In [60]:
marks = {'"', '《', '》', '[', ']', ':', ',', '<', '>', ' ', '(', ')', '?', '/', '.', '∵', '-', '*', '②', '╗', '°', 
         '____', '█', ';', '￥', '#', "'", '·', '①', '------', '!', '|', '='}

In [61]:
def get_answer_list(db_test, y_pred_start, y_pred_end, caseStudy=False):
    def GetAnswer(passage, start, end):
        return passage[start:end + 1]
    
    rankScore = {}
    
    for idx in range(len(db_test)):
        test_details = db_test[idx][-1]
        query_id = str(test_details['query']['query_id'])
        query = test_details['query']['query_text']
        answer = test_details['answer']
        passage = test_details['passage']['passage_text']
        thisMax = -1
        canswer = "NaN**"
        for i in range(setting.passage_len):
            for j in range(i, setting.passage_len):
                if j - i > 9:
                    break
                score = y_pred_start[idx][i] * y_pred_end[idx][j]
                if j - i == 1:
                    score *= 0.1
                if score > thisMax or canswer == "NaN**":
                    temp = GetAnswer(passage, i, j)
                    if temp in marks: continue
                    if temp in query: continue
                    if temp == 'NaN**': continue
                    if len(temp) == 1:
                        a = re.search(pattern='[a-zA-Z\d]', string=temp)
                        try:
                            a.group(0)
                            score *= 0.01
                        except:
                            pass
                    score *= Satisfy(query, temp)
                    if score > thisMax or canswer == "NaN**":
                        thisMax, canswer = score, temp
        rankScore[query_id] = rankScore.get(query_id, {'query': query,
                                                       'answer': answer,
                                                       'answer_rank': []})
        rankScore[query_id]['answer_rank'].append({
                                                    'score': score,
                                                    'answer': canswer,
                                                    'passage': passage
                                                   })
        
    for i, j in rankScore.items():
#         rankScore[i]['answer_rank'] = sorted(j['answer_rank'], key=lambda x: x['score'], reverse=True)
        rankScore[i]['answer_rank'] = get_answer(rankScore[i]['answer_rank'])
        rankScore[i]['pred_ans'] = [item[0] for item in rankScore[i]['answer_rank']]
        
    return rankScore

In [62]:
rank_score = get_answer_list(validation_set_raw, y_pred_start, y_pred_end)

In [63]:
def evaluate(answer_list):
    tr = 0
    tot = len(answer_list)
    for qid, item in answer_list.items():
        print(item['query'], item['answer'], item['pred_ans'])
        answers = qidAnswers.get(str(qid), None)
        t_answer = item['answer']
        ans = item['pred_ans']
        if t_answer == ans[0] or ans[0] in answers:
            tr += 1
            print(t_answer)
            print(ans)
            print('='*20)
#         if t_answer in ans:
#             tr += 1
#             print(t_answer)
#             print(ans)
#             print('='*20)
    print('tr: ', tr)
    print('tr / tot: ', tr / tot)

In [64]:
evaluate(rank_score)

和尚打着一把伞 是一 无法无天 ['-31', '一把伞,', '日', ',是一个', '一个新春,送', '发', '个', '门', '无']
bankindone 印度尼西亚 ['usr', 'u:rupiagh', 'nk:indones', 'zerl', 'wit', 'sia:', '印', 'esia', '属']
裴秀智是哪个组合的 missa组合 [':a韩国美女组合', '手', '智,韩国人', '白', 'sa组', '秀智:女子组合m', '2010年,以mi', '去', '美', '厚', '是个假脸的事实,']
电灯是谁发明 爱迪生 ['英', '学', '终', '美', '般', '是爱迪生发明的:电灯', '王', '发明的', '照', '类', '分']
创立法家学说的是谁 韩非子 ['anx', '下', '韩', '溯', '家不是谁创立的,它只', '第', '原']
千古诗才蓬莱文章建安 李白 ['具楚地方声,', '以', '李', '31:', '章建安骨,中间小', '房子会有很多有', '11,千古诗才,蓬莱', '有', '小']
一个柯南做梦是什么成 南柯一梦 ['就', ']', '来', '剧', '该', '梦一场', '!"柯']
小小的小孩原唱 苏芮 ['的原唱', 'NaN**', '歌', '小小少年(', '小孩》', 'cv5', '独', '妆包,', '清']
结婚十年是什么婚呀 锡婚 ['恭', '叫', '了', '被', '第', '的', ',结婚十年被', '北京爱情故事》.结婚', '单', '年了,']
玉米期货一手多少吨 10吨 ['/吨规定玉', '手是多少:玉米', '怎样的', '10吨', ',最新', '货一手多少吨"', '1600左右.', '动', ',请关注第一黄金网', '多少吨论']
试管婴儿打一影视用语 制作人 ['三国演义》人名):9', '>:试', '仲', '.打一影视用', '蹋粮食(', '谜语:试管婴儿打', '番', '怀', ':y-:u']
孔乙己选自鲁迅的什么 呐喊 ['《风筝》选自《野草', '《风筝》选自《野', '部作品:孔乙', '年', '《呐:孔乙己》选自《', '乙己:鲁迅先生的代表', '戏', '家'

高架逆行扣多少分 12分 ['定:高速道', ',上高', '扣光!12', '吊销驾', ':"司', '予', '"则根']
环县属于甘肃那个市 庆阳市 ['于:甘肃省:东部', '南', '共辖24个行政村', '中', '属于甘肃省庆', '镇西北', '位']
水润草是由什么植物中 肌肤 ['一款很有社', ',污浊感,痒痛', '十', '40元', '污', '不同而:皙美佳人', '被', '后', '弹']
唐朝开国皇帝 李渊 ['高', '大', '继', '朝的开国皇帝和最后一', '唐代开国皇帝.西汉前', 'pt,', '李', '灵州,', '国皇帝!:首页:日志', '家']
西出阳关无故人的阳关 甘肃省 ['甘', '西省咸', '送', '西走廊', '位', ':a:', '走']
凡人修仙传演播作者 忘语 ['桑', '年', '《凡人修', '的', '和', ':忘语:播音:大', '对', '效']
山西省左权县属于哪个 晋中市 ['属于山西省晋中市', '泉', '演:集', '村,22087人', '辖28个行政村,2', '山西哪个', '大', '存', '接']
李超人是谁 李嘉诚 ['外', '女', '外号来看,李超人', '图', '年', '棋', '管"李超人"一再否认', '野', '人"撤']
水浒传什么朝代 北宋 ['施', '小说而已', '是', '明', '十', '的', '朝代:〈水浒传〉的作', '宋']
汉阴县属于哪个市 安康市 ['用', '隶', '养', '县属于安康市,汉', '天', '县.汤', '是', '吧,属', '阴旅游', '化']
披萨哪个国家的 意大利 ['萨来多', '中', '家发明的', '意', '明', '战后回国', '68条韩国', '萨是哪个国家发明的?']
闰年一年多少天 366天 ['年(leap', '365', '366日.', '年怎么计', '含义:', '366天', '2(个)2(天', '退休年', '年有3']
周小栀和林一电影 同桌的你 ['后', '部', '题', '她', '有谁周小栀和林一一起', '《同桌的你》创', '同桌的', '有']
女人生孩子成语是什么 血口喷人 ['穴', '龙', '什么—', '

魔仙女王的扮演者 王慧 ['之', '王慧:饰:魔仙女王', '分', '巴', ':王慧:http:/', '周', '.魔仙', '后']
有顶蘑菇那个叫什么游 超级玛丽 ['《超级', '过', '16-06', '还', '我', '%', '的顶蘑', '就', '超', '见']
牛百叶是牛的哪个胃 瓣胃 ['家', 'ucom:牛腩牛', ',一共有四个胃', 'ucom:牛腩牛百叶', ':,俗作:', '有', '?牛百', '牛的胃吧', '就是蒙牛的肚子吧,']
普洱茶哪个省 云南省 ['因', '验):', '大', '识', '常', '云', '处', '8日起', '50元氛围']
周口师范学院是几本 2本 ['本:周口', '学院是几本呢?我想知', '本科高校.学校始建', '本_周', '所发展建设中的院校,', '院是几本:', '能看不到周口', ',周口师范学院在', '能看不到周口师范', '二']
哈市是哪个省的 黑龙江 ['当', '黑', '然', '还', '尔', '下', '字']
撮土为坟打一字 文 [')', '江', '追', '人', '文', '其', '边', '针']
恋爱邻距离陆蔚萱扮演 莫允雯 ['允', ',更因此有「', ']', '「', '推荐:网友评论', '个人性格啦,', '望']
广州西华路在哪个区 荔湾区 ['州:联系卖家:徐韵涵', '网', '专', '方', '生', '荔', '广州正式启动不动产登', '模', '学']
微微一笑很倾城拍摄学 华东政法大学 ['d:l', '觉', '剧', ':微微一笑', '梭', '青', '小', '校']
世界上最深的湖 贝加尔湖 ['贝', '湖底1', '湖贝加', '位', '0km', '猎', '的湖:湖泊给人一种感', '湖:湖泊给人']
番禺区属于哪个省 广东省 ['属于广州番', '番禺属于', '广', '已', '.番禺', '市', '是广州的一个区', '一', '于:广']
小狮子车标是什么车 标致 ['荣', '哼', '等', ' 宾果,有图有真相哦', '蒙', '个', '架', ';右侧的', '象', '搜']
f字母占几格 3格 ['_gh', '长', '汉语拼音是上三格', '上

gbf是什么游戏 碧蓝幻想 ['网页r', ':]:', '387游', 'oggay "g', '是:3', '最近玩过', '玩家「', '最', ':0个', ',真是', '个', '笑']
四根肋骨骨折定几级伤 10级 ['海南-海口市:咨', '6055', '骨骨折构十级伤残', '骨折;', '骨折了', '我', '前', '说是我的肋', '级伤残']
坦泰尼克号女主角色名 凯特温斯莱特 ['莱特kat', 'kat', '泰坦尼克号女主是:', '泰坦尼克号》女主角:', '泰坦尼克号》女', ',叔叔,外祖父母都', '片', ',叔叔,外祖父母', '泰坦尼克号》', '卡']
夏天爸爸的扮演者 朱佳煜 ['人的战', '演者是', '2005年12月', '施', '侯磊爸', '情', '姑娘,但是夏天的爸爸', '朱', '起', '周']
夏普是哪个国家的品牌 日本 ['会员1', '是日本的品牌,是', '的品牌:夏普是', '官', '日', '啊', '个国家生', '名', '是一个岛国的牌子来', '国家的品牌:']
描写人间情谊的成语 心心相印 ['貌', '情谊的成语:恩重如', '间情谊的成语:', '谊的成语心', '一', '情谊的成语:恩重', '谊的成语:描写人', '的成语:1,一见如', '情投意']
你知道我在等你吗原唱 张洪量 ['是', '演', '5:[', '曲', '原唱:', '林苑的', '就', '手机上继续观看:', ':二维码2小']
跪地求乳打一生肖 羊 ['200', '羊', '会20', '年苦心', '浏览:28010', '欲', '以《生', '书', '性自温']
二战法国总统 戴高乐 ['来', '当', '是', '电', '940年以前法国总', '(记者:卢苏燕)法国', '国傀儡政权是维希', '乐', '总统说']
毛泽东有几个老婆 罗一秀，杨开慧，贺子珍，江青 ['几个妻子:导语:毛', '挂', '氏四修', '第三个妻子.莫名其', '她', '泽东走', '是一个', '泽东和他的妻子儿女(']
南昌市属几线城市 2线 ['一线:广州:深圳:(', '争了 4线 掉线', '是世界十大动感都', '性话题!南昌是二线城', '"是福建旅游的', '

保定哪里的驴肉最正宗 河间 ['火', '了', '定河间', '肉火烧好吃,保定最', '肉火烧品牌店是保定', '肉火烧的标准,莲', '小', '路', '的驴肉比较正宗', '定出名的驴肉火烧,']
江苏海门市属于哪个市 南通市 ['苏省南通市', '南', '于哪个市[惠道', '3日游:', '/we', '海门属', '头', '金', '汕头海门']
昆明好的技校是哪所 中专 ['云', '工作的', '中专学校招生昆明好的', '所——', '云南爱睿尔电脑', '业', '生', '叫专科中的', '业建立']
菏泽是哪个省 山东省 ['度', '-08', '是山东省17地级', '标', '屋', '持', '量', '是哪个省的:你们', '道', '山']
韩红做评委的节目叫什 中国梦之声 ['国', '卫视《', 'NaN**', '中国梦', '明', '评委的综', '少', '骂']
车辆合格证由谁保管 车管所 [',加盖车购费"', ',审验单,机动', '这', '是']
甲午战争是哪一年 1894年 ['1894年', '甲午战争是哪:李慎', ':是哪一年', '***754', '甲午战争1', '被称之', ',辰,', '略战争中,', '午战争可以']
1894年
['1894年', '甲午战争是哪:李慎', ':是哪一年', '***754', '甲午战争1', '被称之', ',辰,', '略战争中,', '午战争可以']
北京首大耳鼻喉医院是 三级 ['最高端', '疗', '喉医院是一所北京', '医院,', '简介:北京首大耳鼻', '首大耳鼻喉医院的详', '首大耳鼻喉医院的', '鼻喉医院.', '鼻喉医院[官', '医院:']
小便刺痛小便不停挂什 泌尿科 ['尿', '检查挂什', '院', '妇科,查白带,', '科', '疾', '列腺疾病:', '感', '泌']
jbl是哪个国家的品 美国 ['响', '美', '飞', '这几个', '曾', '销', '提供各种求解j']
骆天虹是什么电影 夺帅 ['夺', '电影是《夺帅', '新', '话', '5年就开始接拍电视剧', '的', '发', '以', '不']
比喻危险之地带海成语 刀山火海 ['山火海', '刀', '2比喻危',

...

红娘是哪部作品中的人 西厢记 ['"茅庐"本意是指谁的', '的国家是美', '的国家是美国,', '人物?:', '非', '西', '知道你说的红娘', '的形象促进了《西厢', '娘:是哪部作品中', '剧', '是《西厢记》的人物']
迪玛希的老板是谁 尚雯婕 ['谭', '唱', '以', '持带给', '了', '参加活动', '目中唱过', '在', '加']
我国第一位进入太空的 杨利伟 ['王', '空的中', '题', '览', '天', '能', '人', '入太空的宇']
张艺兴陈伟霆赵丽颖的 老九门 ['该', '机', '日前,由', '合', '音', '丽颖陈', '伟霆,', '霆,张']
长沙是哪个省的城市 湖南省 ['者:肾康:长沙是哪', '就', '县', '[:免', '地', '不管哪个', '门', ':长沙', '湖']
安徽省濉溪县属于哪个 淮北市 ['皆', '是', '什', '村隶属', '属于庐', '位于安徽', '属安徽']
风吹树枝摇 折断小枝 枫 ['枝条这', '送', '过:"', '个', '"小树', '就', '木', '上', '树被折', '子', 'k']
不要走 打一字 还 ['小', '最', '面', '干', '个', '便', '还', '趾']
保罗是哪一年的新秀 2005年 ['2):', '2005年', ':球队', ':山猫:', '5:山猫:', '0:保', ':克里斯·保罗', '场', '户', '1:保罗把每个']
死侍扮演者 瑞安雷诺兹 ['.瑞安雷诺兹14岁因', '2016年', '爆', '员', '1位死侍', '韦', '来说,', '里斯,']
史今扮演者 张译 ['nic', '小', '中', '班', '20日左右开机.', '制', '身高:178', '今为什么退']
日《大约在冬季》是谁 齐秦 ['cop', 'igh', '是齐秦演', '专', '歌', '号倾情演唱《大', '00度大为演唱《大', '靠', ':二维码2小时']
电商专业专接本报什么 电子商务专业 ['务', '个', '问考哪', '科', '管理系', '管', '的']
一世纪是多少年 100年 ['少年?年代如何', '100', '国通用的,指

世界上最小的鸟 蜂鸟 ['蜂', '百', '小的鸟:', ',小如', '是', '世界上最小的鸟:说', '小的鸟有', '共', '的鸟,']
杨东煜参加的深圳卫视 超级模仿秀 ['演出;:', '王', '右', '杨东煜在深', '次播放', '中', '签', '曲艺家协会和深圳', '场', '订', '看:二', 'app']
内蒙古巴彦淖尔市新华 临河区 ['不是自', '在', '邮', ',在:内蒙古巴', '果', '如果是', '淖尔市四季']
博白县属于哪个市 玉林市 ['博白是指博白县,县', '广', '图', '截', '西', '都 博', '居', '帮']
名爵汽车是哪个国家的 英国 ['阿', '占地面积', '5概念车', '案', 'g标致', '亿元,占地', '原', '国家的品牌?', '处', '高', '英']
一条彩虹一个气字打一 气贯长虹 ['气贯长虹[解', '气贯长', '2013', '高', '为', '带', '自', '疯', ',疯狂']
武侯祠出师表谁的真迹 岳飞 ['岳', '诸', ':出师', '读', '皇', '侯祠进', '在', '现']
刘诗诗的婚戒戴在哪个 戒指 ['ry:', ',我也首选左手', '面', ':?:大拇指:据', '吴', '是', '只', ',适合你带', ',刘诗']
滕州属于哪个市 枣庄 ['哪里滕州市是山东', '于:山东省:,', '台', '府', '山', '年发展', '该', '枣']
雍正是康熙的第几个儿 第4个 [',康熙皇帝第四子,康', '祖玄烨第四', '第四子', '元17', '马', '生顺序是第十一子:', '康熙六年']
iphonezp是什 港版 ['版', '河', '们', '连', ' :feng/r', 'p是港', 'gb:白色:mg48', 'ome键?传ipho', '港']
哈沙克斯坦首都 阿斯塔纳 ['是', '的', '阿', ':12', 'ma-a', '萨克斯', ',吉尔', '吉', ',成为', '会', '景']
让一切随风原唱 钟镇涛 ['镇', '由', '谭', '风》演', '随风:', '典']
常州是哪个省的 江苏省 ['咱', '(江苏省地级市),所', '帅', 

高尔基是哪个国家的 前苏联 [')', '无', '俄', '历', '时', '广', '后']
伊利的创始人是谁 郑俊怀 ['郑俊怀', '利牛奶的创始人是谁?', ':伊利创', '奶的创', '伊利创', '—']
郑俊怀
['郑俊怀', '利牛奶的创始人是谁?', ':伊利创', '奶的创', '伊利创', '—']
闭关锁国是哪个朝代 清朝 ['清', '时', '辑', '答', '二', '元', '明']
wey汽车是哪国的车 中国 ['魏', ':精华:', 'ey:', '马', '?:精华:', '长', '城', ':24小时全国销售', '金', '的事情', ',降低了冲击']
顾则霖林冉冉小说叫什 小鹿 ['读:林冉冉', '则霖小说名字叫什么?', '林冉冉顾则霖的小', '叫做《契婚》,作者是', '婚', '载和阅', '主', '冉冉顾', '角林冉', '福', '己的幸福']
自爆卡车是哪个国家 利比亚 ['不', '卡车 ', '升', '好', '卡车:', '袭', '维', '星', '利']
巨野县属于哪个市 菏泽市 ['野县在', '东', '山', '中', '清朝,巨野', '耕', '公', '尤', '农', '济']
华安里社区属于哪个区 江汉区 ['汉口火', '武汉:江', '旅', '电', '隶属于', '附', '武', '并']
中国人民解放军火箭军 2015年 ['2015', '2015年12月3', '词:2015', '国人民解放军陆军领导', '人民解放军火箭军是中', '解放军火箭军成立于', '就是在"二炮"的基', '国人民解放军:火', '是']
2015年
['2015', '2015年12月3', '词:2015', '国人民解放军陆军领导', '人民解放军火箭军是中', '解放军火箭军成立于', '就是在"二炮"的基', '国人民解放军:火', '是']
卡地亚是哪个国家的品 法国 ['属', '(co', '品牌:卡地亚', '华', '度', '亚的品牌历史:', '哪个国家的品牌', '个国家的品牌,']
四万万同胞是多少人 400000000人 ['万=4', '就是四', '"四万万同胞"具', '400', '40000', '万就是四', '4亿', 

麻城在哪个省 湖北省 ['湖', '架', '城三中,虽然管得严,', '位', '三', '把', '元.麻', '2016', '说麻城']
福建省光泽县是哪个市 南平市 ['在', '福建省光泽县李', '询', '政', '位', '县在华东地区算是经济', '县二一', '验', '动']
夏朝持续了多久 400年 ['朝(约为前20', '续约40', '朝(约前', '皇帝的称谓', '了47', '亡']
围魏救赵主人公是谁 孙膑 ['的意思', '救赵的', '膑:围', '卒年不详,中国', '一', '找', '思', '要', '这', '魏救赵的']
梦驼铃原唱 费玉清 [':许冠杰]:04', '铃是台湾外省人的思乡', '费', '龙']
coo什么职位 首席运营官 ['代', '活动的人', '满', 'cer', '织体系', '一年的', '思这个是一', '公', '制', '全']
中国人民解放军海军成 1949年 ['ates:6', '民解放军海军19', '部', '中国空军成立于1', '民解放军海军成立60', '人民解放军海军成立', '国人民解放军海军航', '军海军成立于1949']
克拉玛依之歌原唱 吕文科 ['《克拉玛', '面', '957年', '之:简介:co', '多', '古', '吕', '依之歌—刘和刚[高']
读完高中读大专要几年 3年 ['3年中专了', '标签:今年', '满', '也就是5年', '需要读几年?读出来是', ',一般是五年', '需要三年:大', '年咯,读完高中读大专']
遂宁火车站属哪个区 船山区 ['店', ':遂宁市船山区', '前', '开', '包', '后', '交查询:百度:', '的', '向', '路', '车站广告媒体']
晚上11点出生是什么 子时 ['27日', '年', '亥', '凌', '时', '子', '先']
彳闺寺是什么成语 待字闺中 ['待', '韵', '关', '相', '上', '穿', '的', '逢']
纳智捷是那个国家的品 中国 ['司', '纳智捷定位于', '种', '是哪个国家', '产车种', '性化的', '说', '中', '3年设立裕隆汽车制造', '我国台']
身份证尺寸 85.6mm×54.0mm×

蒹葭是什么植物 芦苇 ['名', ',指芦荻,', '芦', '意', '诗也是《蒹葭', '那', '丽', '诗', '风《蒹', '粗']
山城是我国哪个城市的 重庆 ['-古~', '是重庆的别称,其实山', '号:山城是', '省', '西', '下', '楼', '一', '青']
黑龙江庆安县属于哪个 绥化市 ['河', '属于黑', '次:(', '气', '是', '候', '县属于半山区.地形像', '于绥化东北64', '公']
秘鲁的首都 利马 ['首都是', '吴', '尼', '都是利马(lima)', '流', '于', '[青蛙汁]秘鲁', '哈瓦那', '中']
洪金宝逃狱电影 火烧岛 ['应该是《', '敲', '福', '2017-01-0', '《火烧', '金宝刚', '02:', '不', '火', '谈']
济南祝舜路在哪个区 历城区 ['祝舜路与东晨大街交', ':30', '都', '询', 'ngz', '社', '市', ',:,', '破']
征信贷款记录多久消除 5年 ['2009年', '久消除?我现在想贷', '取', '录都是5', '问一下', '款,征', '才']
带有襟肘的成语 捉襟见肘 ['捉', '案', '哪些捉襟露肘', '见', '是']
中国举办亚洲杯是哪年 2004年 ['2004', '伊', '仍', '杯的经', '种', '界', '第', '家门口', '阵']
2004年
['2004', '伊', '仍', '杯的经', '种', '界', '第', '家门口', '阵']
洪湖水浪打浪原唱 王玉珍 ['王', '首经典老歌,', '湖北歌剧团).', '曲,1', '打浪(王玉珍原', 'NaN**', '按', '睡', '可进行拖动:洪湖水']
gmc是什么牌子 通用汽车公司 ['空', '挑', '用旗下的商务车品牌,', '吉', '供', '商务之星', '起来几', '美']
辣木籽一天吃几粒 辣木籽一天吃10粒 ['吃几粒合适', '木籽一天吃几粒好', '一天吃10粒', '吃几粒,不同情况使用', '3粒辣木籽:1-2粒', '肠胃,促', '好', '一般都是空腹吃,', '二至四粒,一星期', '你', '先吃两粒一周左']
双耳刀几画 2画 

In [37]:
keys = list(rank_score.keys())
print(len(keys))

251


In [38]:
print(rank_score[keys[78]])

{'answer_rank': [{'score': 0.001650687, 'answer': '手游<'}, {'score': 0.0016470924, 'answer': '手游巧思猜灯谜答'}, {'score': 0.0016205538, 'answer': '手游巧思猜灯谜活'}, {'score': 0.0016172873, 'answer': '手游元'}, {'score': 0.0016159853, 'answer': '3>,'}, {'score': 0.0015866966, 'answer': '手游巧思猜灯谜所'}, {'score': 0.0015531802, 'answer': '玩游戏'}, {'score': 0.0015147683, 'answer': '源:齐'}, {'score': 0.0015084198, 'answer': '手游巧思猜灯谜活动'}, {'score': 0.0014134306, 'answer': '西为玩家'}], 'answer': '元宵节', 'pred_ans': ['手游<', '手游巧思猜灯谜答', '手游巧思猜灯谜活', '手游元', '3>,', '手游巧思猜灯谜所', '玩游戏', '源:齐', '手游巧思猜灯谜活动', '西为玩家'], 'query': '梦幻西游手游巧思猜灯谜'}


In [50]:
cnt = 0
for i, j in rank_score.items():
    cnt += 1
    if cnt > 10:
        break
    print(i, j)

12790 {'pred_ans': ['[新东', '0', '无', '迎', '伞,是一个什么成', '最少?:查看', '楚', '·'], 'answer': '无法无天', 'answer_rank': [['[新东', 2.241257977728627e-62], ['0', 2.9465574768631064e-65], ['无', 1.5046364494094624e-65], ['迎', 1.3692264297644458e-65], ['伞,是一个什么成', 1.154219788000716e-65], ['最少?:查看', 1.0655993020154001e-65], ['楚', 1.0655993020154001e-65], ['·', 9.716865868212905e-66]], 'query': '和尚打着一把伞 是一'}
12863 {'pred_ans': ['腾', '不', '设', 'p', '序', '讯!', '微信互动营销,', '这个应用的,但', '听'], 'answer': '腾讯', 'answer_rank': [['腾', 4.149165693992927e-64], ['不', 9.226057606498172e-65], ['设', 9.049642468474842e-65], ['p', 4.306770419994789e-65], ['序', 2.473036226782358e-65], ['讯!', 1.01138335676258e-65], ['微信互动营销,', 6.663806669048271e-66], ['这个应用的,但', 6.663806669048271e-66], ['听', 6.319674095269869e-67]], 'query': '微信是哪个公司开发的'}
10817 {'pred_ans': ['1994年10月10', 's:a', '坦', '秀智:miss', '组合miss:a', '去', '白领ls恩乔三个', '见', '2', '郑'], 'answer': 'missa组合', 'answer_rank': [['1994年10月10', 3.0501966998055e-60], ['s:a', 3.82168